In [8]:
import cv2
import numpy as np

In [9]:
# 创建卡尔曼滤波器
kf = cv2.KalmanFilter(4, 2)

# 初始化状态估计值和协方差矩阵
kf.statePost = np.array([0, 0, 0, 0], dtype=np.float32)
kf.errorCovPost = np.eye(4, dtype=np.float32)

# 设置卡尔曼滤波器参数
dt = 1.0  # 采样间隔
kf.transitionMatrix = np.array([[1, 0, dt, 0],
                                [0, 1, 0, dt],
                                [0, 0, 1, 0],
                                [0, 0, 0, 1]], dtype=np.float32)
kf.processNoiseCov = np.array([[1, 0, 0, 0],
                               [0, 1, 0, 0],
                               [0, 0, 1, 0],
                               [0, 0, 0, 1]], dtype=np.float32) * 0.0001
kf.measurementMatrix = np.array([[1, 0, 0, 0],
                                 [0, 1, 0, 0]], dtype=np.float32)
kf.measurementNoiseCov = np.array([[1, 0],
                                   [0, 1]], dtype=np.float32) * 1

In [11]:
# 打开摄像头
cap = cv2.VideoCapture(0)

# 读取一帧图像
ret, frame = cap.read()

# 将图像转换为灰度图
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

# 检测目标，提取目标的中心点坐标
x, y, w, h = cv2.selectROI("SelectROI", frame, fromCenter=False, showCrosshair=True)

while True:
    # 读取一帧图像
    ret, frame = cap.read()
    if not ret:
        break

    # 将图像转换为灰度图
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 检测目标，提取目标的中心点坐标
   
    cx, cy = x + w/2, y + h/2

    # 对中心点坐标进行卡尔曼滤波
    prediction = kf.predict()
    measurement = np.array([cx, cy], dtype=np.float32)
    kf.correct(measurement)

    # 绘制目标位置（绿）和卡尔曼滤波器预测的位置（红）
    cv2.circle(frame, (int(cx), int(cy)), 5, (0, 255, 0), -1)
    cv2.circle(frame, (int(prediction[0]), int(prediction[1])), 5, (0, 0, 255), -1)

    # 显示结果
    cv2.imshow("Kalman Filter", frame)

    # 按下ESC键退出
    if cv2.waitKey(1) == 27:
        break

# 释放摄像头并关闭窗口
cap.release()
cv2.destroyAllWindows()
